# PyBullet을 이용한 진자(Pendulum) 강화학습 환경

MATLAB의 진자 예제와 유사한 강화학습 환경을 PyBullet으로 구성

- **상태**: `[cos(theta), sin(theta), theta_dot]`
- **행동**: 진자 관절에 가하는 토크 (-2.0 ~ 2.0)
- **보상**: 진자가 위로 설수록, 각속도와 토크가 작을수록 높은 보상 획득.

### 1단계: URDF 파일 생성 (Code)

Jupyter Notebook의 매직 명령어 %%writefile을 사용하면 현재 디렉토리에 바로 파일을 생성할 수 있어 편리하다고 함!

In [ ]:
%%writefile models/pendulum.urdf
<?xml version="1.0"?>
<robot name="pendulum">
  <link name="base"/>

  <link name="pole">
    <inertial>
      <mass value="1.0"/>
      <inertia ixx="0.083" ixy="0.0" ixz="0.0" iyy="0.083" iyz="0.0" izz="0.005"/>
    </inertial>
    <visual>
      <geometry>
        <cylinder length="1" radius="0.05"/>
      </geometry>
      <material name="red">
          <color rgba="1 0.2 0.2 1"/>
      </material>
    </visual>
    <collision>
      <geometry>
        <cylinder length="1" radius="0.05"/>
      </geometry>
    </collision>
  </link>

  <joint name="hinge" type="revolute">
    <parent link="base"/>
    <child link="pole"/>
    <axis xyz="0 1 0"/>
    <limit effort="3" lower="-3.1415" upper="3.1415" velocity="15"/>
    <origin rpy="0 0 0" xyz="0 0 0.5"/>
  </joint>

</robot>

### 2단계: 라이브러리 임포트 (Code)

환경 구성과 테스트에 필요한 라이브러리들을 불러오기

In [2]:
import gymnasium as gym
from gymnasium import spaces
import pybullet as p
import pybullet_data
import numpy as np
import cv2
import time
import os

print("라이브러리 임포트 완료!")

라이브러리 임포트 완료!


## 3단계: 시뮬 구성하기

### 3-1 막대와 바닥과 중력 
우선, 내가 만든 urdf의 조인트에 대해서 정보를 좀 얻어오자.

In [3]:
URDF_PATH = "models/pendulum.urdf" # 실제 URDF 파일 경로

try:
    client = p.connect(p.DIRECT)
    p.setAdditionalSearchPath(pybullet_data.getDataPath())
    
    robot_id = p.loadURDF(URDF_PATH)

    # 1. 전체 조인트 개수 확인
    num_joints = p.getNumJoints(robot_id)
    print(f"'{URDF_PATH}'에 총 {num_joints}개의 조인트가 있습니다.")
    print("-" * 40)

    # 2. 각 조인트의 정보 출력
    for i in range(num_joints):
        joint_info = p.getJointInfo(robot_id, i)
        
        joint_index = joint_info[0]
        joint_name = joint_info[1].decode('utf-8') # byte string을 일반 string으로 변환
        joint_type = joint_info[2]
        child_link_name = joint_info[12].decode('utf-8')
        
        print(f"  인덱스 (Index): {joint_index}")
        print(f"  이름 (Name): {joint_name}")
        print(f"  타입 (Type): {joint_type} (REVOLUTE=0, FIXED=4 등)")
        print(f"  연결된 링크 (child Link): {child_link_name}")
        print("-" * 40)

finally:
    if p.isConnected():
        p.disconnect()

'models/pendulum.urdf'에 총 2개의 조인트가 있습니다.
----------------------------------------
  인덱스 (Index): 0
  이름 (Name): fixed_base
  타입 (Type): 4 (REVOLUTE=0, FIXED=4 등)
  연결된 링크 (child Link): base
----------------------------------------
  인덱스 (Index): 1
  이름 (Name): hinge
  타입 (Type): 0 (REVOLUTE=0, FIXED=4 등)
  연결된 링크 (child Link): pole
----------------------------------------


In [4]:
width, height = 640, 480
frames = []

p.connect(p.DIRECT)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.loadURDF("plane.urdf")
p.setGravity(0, 0, -9.81)
urdf_path = "/models/stick.urdf"
print(urdf_path)

pendulum_id = p.loadURDF(urdf_path, basePosition=[0, 0, 1], useFixedBase=False)

for step in range(240*5):
    p.stepSimulation()

    img = p.getCameraImage(width, height, renderer=p.ER_TINY_RENDERER)
    rgb = np.array(img[2]).reshape((height, width, 4))[:,:,:3]
    frames.append(rgb)

    if step % 100 == 0:
        print(step)
        
video = cv2.VideoWriter('video/test.mp4', cv2.VideoWriter.fourcc(*'avc1'), 20.0, (width, height))
for frame in frames:
    video.write(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))
video.release()

p.disconnect()

/models/stick.urdf
0
100
200
300
400
500
600
700
800
900
1000
1100


[ERROR:0@53.201] global cap_ffmpeg_impl.hpp:3207 open Could not find encoder for codec_id=27, error: Encoder not found
[ERROR:0@53.201] global cap_ffmpeg_impl.hpp:3285 open VIDEOIO/FFMPEG: Failed to initialize VideoWriter


이걸 돌리고 나면 GIF 파일로 확인할 수 있는 것처럼 막대가 낙하하고, 바닥에 부딪혀 넘어지는 것이 확인되어야 한다.

![stick_plane_gravity](docs/step_3_1.gif) 

### 3-2 고정되고, 회전하는 막대
우선, pybullet에서 지원하는 기능을 통해서 카메라 시점을 좀 움직여보자. 

In [6]:
# 카메라 시점 정의하기
cam_eye_pos = [0, 3, 1.5]      # 카메라 위치
cam_target_pos = [0, 0, 1]     # 바라볼 목표 지점
cam_up_vector = [0, 0, 1]      # 카메라의 위쪽 방향

# 2. 뷰 행렬과 투영 행렬 계산
view_matrix = p.computeViewMatrix(
    cameraEyePosition=cam_eye_pos,
    cameraTargetPosition=cam_target_pos,
    cameraUpVector=cam_up_vector
)

projection_matrix = p.computeProjectionMatrixFOV(
    fov=60.0,
    aspect=float(width) / height,
    nearVal=0.1,
    farVal=100.0
)

그리고 이건 설정한 시점과 만든 urdf를 가지고 pybullet 환경을 만드는 것이다.

목적: inverted pendulum 환경은 중력이 제대로 반영된 환경인가? inverted pendulm 객체는 올바르게 설계되었는가?

In [8]:
width, height = 640, 480
frames = []

p.connect(p.GUI)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.loadURDF("plane.urdf")
p.setGravity(0, 0, -9.81)
urdf_path = "models/pendulum.urdf"
print(urdf_path)

poleStartPos = [0,0,0]
poleStartOrientation = p.getQuaternionFromEuler([0,0,0])
pendulum_id = p.loadURDF(urdf_path, basePosition=poleStartPos, baseOrientation=poleStartOrientation, useFixedBase=True)

initial_angle = np.pi*(1/3)
p.resetJointState(
    bodyUniqueId=pendulum_id,
    jointIndex=1,
    targetValue=initial_angle
)
print(f"진자를 {np.rad2deg(initial_angle):.0f}도 위치에서 시작")

p.setJointMotorControl2(
        bodyUniqueId=pendulum_id,
        jointIndex=1,
        controlMode=p.VELOCITY_CONTROL,
        force=0
    )

for step in range(240*10):
    p.stepSimulation()

    img = p.getCameraImage(width, height, 
                           viewMatrix=view_matrix,
                           projectionMatrix=projection_matrix,
                           renderer=p.ER_TINY_RENDERER)
    rgb = np.array(img[2]).reshape((height, width, 4))[:,:,:3]
    frames.append(rgb)

    if step % 100 == 0:
        print(step)

video = cv2.VideoWriter('video/test.mp4', cv2.VideoWriter.fourcc(*'avc1'), 20.0, (width, height))
for frame in frames:
    video.write(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))
video.release()

p.disconnect()

startThreads creating 1 threads.
starting thread 0
models/pendulum.urdf
진자를 60도 위치에서 시작
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=NVIDIA Corporation
GL_RENDERER=NVIDIA GeForce RTX 4080 SUPER/PCIe/SSE2
GL_VERSION=3.3.0 NVIDIA 580.65.06
GL_SHADING_LANGUAGE_VERSION=3.30 NVIDIA via Cg compiler
pthread_getconcurrency()=0
Version = 3.3.0 NVIDIA 580.65.06
Vendor = NVIDIA Corporation
Renderer = NVIDIA GeForce RTX 4080 SUPER/PCIe/SSE2
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = NVIDIA Corporation
ven = NVIDIA Corporation
0
100
200
300
400
500
600


: 

직접 좌표계를 그리니까 훨 낫다. 

![stick_plane_gravity](docs/step_3_2_with_weight.gif)
![stick_plane_gravity](docs/step_3_2_wo_weight.gif) 

보니까, 무게추가 꼭 필요할 줄 알았는데 urdf를 잘 작성하고 나니 큰 문제는 아니었군.
그래프 뽑는 것도 함수로 만들어놓으면 편하겠다! 

### 3-3단계: 클래스로 PyBullet 진자 환경 정의하기
여기에 클래스 작성하기

In [ ]:
import gymnasium as gym
from gymnasium import spaces
import pybullet as p
import pybullet_data
import numpy as np
import cv2
import time
import os

print("라이브러리 임포트 완료!")

class PendulumEnv(gym.Env):
    metadata = {"render_modes":["human", "rgb_array"]}
    def __init__(self, render_mode="human", video_path=None):
        super(PendulumEnv, self).__init__()
        
        self.render_mode = render_mode
        if self.render_mode == 'human':
            try:
                p.disconnect()
            except p.error:
                pass
            self.client = p.connect(p.GUI)
        else:
            self.client = p.connect(p.DIRECT)

        self.g = 9.81
        self.maxTorque = 2.0

        # 시뮬 동영상 기록 (1) 카메라 시점 정의
        cam_eye_pos = [0, 3, 1.5]      # 카메라 위치
        cam_target_pos = [0, 0, 1]     # 바라볼 목표 지점
        cam_up_vector = [0, 0, 1]      # 카메라의 위쪽 방향
        self._render_width=640
        self._render_height=480

        # 시뮬 동영상 기록 (2) 뷰 행렬과 투영 행렬 계산
        self.view_matrix = p.computeViewMatrix(
            cameraEyePosition=cam_eye_pos,
            cameraTargetPosition=cam_target_pos,
            cameraUpVector=cam_up_vector
        )
        self.projection_matrix = p.computeProjectionMatrixFOV(
            fov=60.0,
            aspect=float(self._render_width) / self._render_height,
            nearVal=0.1,
            farVal=100.0
        )
        self.frames = []
        self.videoPath = video_path
        
        # 모델 불러오기 
        p.setAdditionalSearchPath(pybullet_data.getDataPath())
        p.loadURDF("plane.urdf")
        p.setGravity(0, 0, -self.g)

        modelPath="models/pendulum.urdf"
        print(f"model path: {modelPath}")
        modelStartPos = [0, 0, 0]
        modelStartOri = p.getQuaternionFromEuler([0, 0, 0])
        self.modelId = p.loadURDF(modelPath, basePosition=modelStartPos, baseOrientation=modelStartOri, useFixedBase=True)

        # 연속 행동 공간.
        # self.actionSpace = spaces.Box(low=-self.maxTorque, high=self.maxTorque, shape=(1,), dtype=np.float32)

        # 이산 행동 공간 
        self.actionSpace = spaces.Discrete(3, start=-1)
        
        # 관측 공간 정의하기.
        high = np.array([1.0, 1.0, 15.0], dtype=np.float32) # 여기에서 최대 속력 15 괜춘?
        self.observationSpace=spaces.Box(low=-high, high=high, dtype=np.float32)

        self.stepCounter = 0
    
    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.stepCounter = 0
        # self.frames = []

        # 모델 관절 초기화 
        initialAngle= np.pi*(1) 
        initVelocity = 0
        # initialAngle= self.np_random.uniform(low=-np.pi, high=np.pi) 
        # initVelocity = self.np_random.uniform(low=-1.0, high=1.0)

        p.resetJointState(
            bodyUniqueId=self.modelId,
            jointIndex=1,
            targetValue=initialAngle,
            targetVelocity=initVelocity
        )
        print(f"진자를 {np.rad2deg(initialAngle):.0f}도 위치에서 시작")

        p.setJointMotorControl2(
            bodyUniqueId=self.modelId,
            jointIndex=1,
            controlMode=p.VELOCITY_CONTROL,
            force=0
        )
        
        return self._get_obs(), {}
    
    def step(self, action):
        torque = float(action) * self.maxTorque
        # print(f"torque value: {torque}")    

        p.setJointMotorControl2(
            bodyUniqueId=self.modelId,
            jointIndex=1,
            controlMode=p.TORQUE_CONTROL,
            force=torque
        )

        p.stepSimulation()

        img = p.getCameraImage(self._render_width, self._render_height, 
                           viewMatrix=self.view_matrix,
                           projectionMatrix=self.projection_matrix,
                           renderer=p.ER_TINY_RENDERER)
        rgb = np.array(img[2]).reshape((self._render_height, self._render_width, 4))[:,:,:3]
        self.frames.append(rgb)

        if self.stepCounter % 100 == 0:
            print(self.stepCounter)

        obs=self._get_obs()
        reward=self._calculate_reward(obs, action)

        self.stepCounter +=1
        truncated = bool(self.stepCounter >= 200)
        terminated = False

        return obs, reward, terminated, truncated, {}

    def _get_obs(self):
        jointState = p.getJointState(self.modelId, 1)
        theta, theta_dot = jointState[0], jointState[1]
        theta = (theta + np.pi) % (2*np.pi) - np.pi # 각도 정규화
        
        return np.array([np.cos(theta), np.sin(theta), theta_dot], dtype=np.float32)
    
    def _calculate_reward(self, obs, action):
        cos_theta, sin_theta, theta_dot = obs
        theta = np.arctan2(sin_theta, cos_theta)
        torque= float(action)
        reward = -(theta**2 + 0.1 *theta_dot**2 + 0.001*torque**2)

        return reward 
    
    def render(self):
        pass

    def close(self):
        videoFile = os.path.join(self.videoPath, "test.mp4")
        print(f"video recording start ... {videoFile}")
        video = cv2.VideoWriter(videoFile, cv2.VideoWriter.fourcc(*'mp4v'), 30, (self._render_width, self._render_height))
        for frame in self.frames:
            video.write(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))
        video.release()

        p.disconnect(self.client)

print("클래스 정의 완료!")

라이브러리 임포트 완료!
클래스 정의 완료!


### 3-4단계: 환경 테스트 - 클래스

정의된 환경을 실제로 생성하고, 무작위 행동을 주면서 어떻게 동작하는지 테스트
우선, 불러온 펜듈럼이 시뮬에서 제대로 돌아가는지나 보자. 

In [ ]:
env = PendulumEnv(render_mode="human", video_path="video")
obs, info = env.reset()
totalReward = 0

print("Env start ... ")

for i in range(240*3):
    action = env.actionSpace.sample()
    
    obs, reward, terminated, truncated, info = env.step(action)
    totalReward += reward

    # time.sleep(1./240.)
print(f"{i+1}st episode")

print("\n Env close")
env.close()

다행히 잘 된다. 

![alt text](docs/step_5_1.gif)